<a href="https://colab.research.google.com/github/amoosebitmymom/MUSDB/blob/main/MUSDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metadata


consists of information necessary for the exection of the program, must always be run before doing changes in the code

In [ ]:
""" installing the mudsb package, which includes 7 second song segments as stem files """
!pip install musdb

     |████████████████████████████████| 963 kB 45.7 MB/s 


In [ ]:
import os
import sys
import time
import musdb
import pickle
import IPython
import numpy as np
import pandas as pd
import librosa as lb
import stempeg as sp
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from matplotlib import cm
from sklearn import metrics 
from librosa import display
from IPython import display
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split 

MUSDB 2018 Dataset <br>
The musdb18 consists of 150 songs of different styles along with the images of<br> their constitutive objects.<br>

musdb18 contains two folders, a folder with a training set: "train", composed <br>of 100 songs, and a folder with a test set: "test", composed of 50 songs. <br>Supervised approaches should be trained on the training set and tested on both<br> sets.

All files from the musdb18 dataset are encoded in the Native Instruments stems <br>format (.mp4). It is a multitrack format composed of 5 stereo streams, each one<br> encoded in AAC @256kbps. These signals correspond to:<br>

0 - The mixture,<br>
1 - The drums,<br>
2 - The bass,<br>
3 - The rest of the accompaniment,<br>
4 - The vocals.<br>

For each file, the mixture correspond to the sum of all the signals. All <br>signals are stereophonic and encoded at 44.1kHz.<br>

The data from musdb18 is composed of several different sources:

100 tracks are taken from the DSD100 dataset, which is itself derived from The<br> 'Mixing Secrets' Free Multitrack Download Library. Please refer to this <br>original resource for any question regarding your rights on your use of the <br>DSD100 data.<br>

46 tracks are taken from the MedleyDB licensed under Creative Commons (BY-NC-SA 4.0).<br>

2 tracks were kindly provided by Native Instruments originally part of their stems pack.<br>

2 tracks a from from the Canadian rock band The Easton Ellises as part of the <br>heise stems remix competition, licensed under Creative Commons (BY-NC-SA 3.0).



In [ ]:
""" 
a list of all the song names in the dataset -
0 - 99: train folder
100 - 149 : test folder 
"""
song_names = [
"A Classic Education - NightOwl.stem",
"Actions - Devil's Words.stem",
"Actions - One Minute Smile.stem",
"Actions - South Of The Water.stem",
"Aimee Norwich - Child.stem",
"Alexander Ross - Goodbye Bolero.stem",
"Alexander Ross - Velvet Curtain.stem",
"Angela Thomas Wade - Milk Cow Blues.stem",
"ANiMAL - Clinic A.stem",
"ANiMAL - Easy Tiger.stem",
"ANiMAL - Rockshow.stem",
"Atlantis Bound - It Was My Fault For Waiting.stem",
"Auctioneer - Our Future Faces.stem",
"AvaLuna - Waterduct.stem",
"BigTroubles - Phantom.stem",
"Bill Chudziak - Children Of No-one.stem",
"Black Bloc - If You Want Success.stem",
"Celestial Shore - Die For Us.stem",
"Chris Durban - Celebrate.stem",
"Clara Berry And Wooldog - Air Traffic.stem",
"Clara Berry And Wooldog - Stella.stem",
"Clara Berry And Wooldog - Waltz For My Victims.stem",
"Cnoc An Tursa - Bannockburn.stem",
"Creepoid - OldTree.stem",
"Dark Ride - Burning Bridges.stem",
"Dreamers Of The Ghetto - Heavy Love.stem",
"Drumtracks - Ghost Bitch.stem",
"Faces On Film - Waiting For Ga.stem",
"Fergessen - Back From The Start.stem",
"Fergessen - Nos Palpitants.stem",
"Fergessen - The Wind.stem",
"Flags - 54.stem",
"Giselle - Moss.stem",
"Grants - PunchDrunk.stem",
"Helado Negro - Mitad Del Mundo.stem",
"Hezekiah Jones - Borrowed Heart.stem",
"Hollow Ground - Left Blind.stem",
"Hop Along - Sister Cities.stem",
"Invisible Familiars - Disturbing Wildlife.stem",
"James May - All Souls Moon.stem",
"James May - Dont Let Go.stem",
"James May - If You Say.stem",
"James May - On The Line.stem",
"Jay Menon - Through My Eyes.stem",
"Johnny Lokke - Promises & Lies.stem",
"Johnny Lokke - Whisper To A Scream.stem",
"Jokers, Jacks & Kings - Sea Of Leaves.stem",
"Leaf - Come Around.stem",
"Leaf - Summerghost.stem",
"Leaf - Wicked.stem",
"Lushlife - Toynbee Suite.stem",
"Matthew Entwistle - Dont You Ever.stem",
"Meaxic - Take A Step.stem",
"Meaxic - You Listen.stem",
"Music Delta - 80s Rock.stem",
"Music Delta - Beatles.stem",
"Music Delta - Britpop.stem",
"Music Delta - Country1.stem",
"Music Delta - Country2.stem",
"Music Delta - Disco.stem",
"Music Delta - Gospel.stem",
"Music Delta - Grunge.stem",
"Music Delta - Hendrix.stem",
"Music Delta - Punk.stem",
"Music Delta - Reggae.stem",
"Music Delta - Rock.stem",
"Music Delta - Rockabilly.stem",
"Night Panther - Fire.stem",
"North To Alaska - All The Same.stem",
"Patrick Talbot - A Reason To Leave.stem",
"Patrick Talbot - Set Me Free.stem",
"Phre The Eon - Everybody's Falling Apart.stem",
"Port St Willow - Stay Even.stem",
"Remember December - C U Next Time.stem",
"Secret Mountains - High Horse.stem",
"Skelpolu - Human Mistakes.stem",
"Skelpolu - Together Alone.stem",
"Snowmine - Curfews.stem",
"Spike Mullings - Mike's Sulking.stem",
"St Vitus - Word Gets Around.stem",
"Steven Clark - Bounty.stem",
"Strand Of Oaks - Spacestation.stem",
"Sweet Lights - You Let Me Down.stem",
"Swinging Steaks - Lost My Way.stem",
"The Districts - Vermont.stem",
"The Long Wait - Back Home To Blue.stem",
"The Scarlet Brand - Les Fleurs Du Mal.stem",
"The So So Glos - Emergency.stem",
"The Wrong'Uns - Rothko.stem",
"Tim Taler - Stalker.stem",
"Titanium - Haunted Age.stem",
"Traffic Experiment - Once More (With Feeling).stem",
"Traffic Experiment - Sirens.stem",
"Triviul - Angelsaint.stem",
"Triviul - Dorothy.stem",
"Voelund - Comfort Lives In Belief.stem",
"Wall Of Death - Femme.stem",
"Young Griffo - Blood To Bone.stem",
"Young Griffo - Facade.stem",
"Young Griffo - Pennies.stem",


"Al James - Schoolboy Facination.stem",
"AM Contra - Heart Peripheral.stem",
"Angels In Amplifiers - I'm Alright.stem",
"Arise - Run Run Run.stem",
"Ben Carrigan - We'll Talk About It All Tonight.stem",
"BKS - Bulldozer.stem",
"BKS - Too Much.stem",
"Bobby Nobody - Stitch Up.stem",
"Buitraker - Revo X.stem",
"Carlos Gonzalez - A Place For Us.stem",
"Cristina Vane - So Easy.stem",
"Detsky Sad - Walkie Talkie.stem",
"Enda Reilly - Cur An Long Ag Seol.stem",
"Forkupines - Semantics.stem",
"Georgia Wonder - Siren.stem",
"Girls Under Glass - We Feel Alright.stem",
"Hollow Ground - Ill Fate.stem",
"James Elder & Mark M Thompson - The English Actor.stem",
"Juliet's Rescue - Heartbeats.stem",
"Little Chicago's Finest - My Own.stem",
"Louis Cressy Band - Good Time.stem",
"Lyndsey Ollard - Catching Up.stem",
"M.E.R.C. Music - Knockout.stem",
"Moosmusic - Big Dummy Shake.stem",
"Motor Tapes - Shore.stem",
"Mu - Too Bright.stem",
"Nerve 9 - Pray For The Rain.stem",
"PR - Happy Daze.stem",
"PR - Oh No.stem",
"Punkdisco - Oral Hygiene.stem",
"Raft Monk - Tiring.stem",
"Sambasevam Shanmugam - Kaathaadi.stem",
"Secretariat - Borderline.stem",
"Secretariat - Over The Top.stem",
"Side Effects Project - Sing With Me.stem",
"Signe Jakobsen - What Have You Done To Me.stem",
"Skelpolu - Resurrection.stem",
"Speak Softly - Broken Man.stem",
"Speak Softly - Like Horses.stem",
"The Doppler Shift - Atrophy.stem",
"The Easton Ellises - Falcon 69.stem",
"The Easton Ellises (Baumi) - SDRNR.stem",
"The Long Wait - Dark Horses.stem",
"The Mountaineering Club - Mallory.stem",
"The Sunshine Garcia Band - For I Am The Moon.stem",
"Timboz - Pony.stem",
"Tom McKenzie - Directions.stem",
"Triviul feat. The Fiend - Widow.stem",
"We Fell From The Sky - Not You.stem",
"Zeno - Signs.stem"
]

In [ ]:
path = "/content/drive/MyDrive/Data/MUSDB/" # basic directory, which includes two folders (train, test) and within them the songs 

# Data downloading

downloads the song segments, no need to run because they are saved on google drive

In [ ]:
mus = musdb.DB(root="/content/drive/MyDrive/Data/MUSDB", download=True) 

# Data verification

verifys all the data exists and seprates the data into its corresponding folders

In [ ]:
"""
a function which checks if a given file exists. If it does not, it saves its name

path = the basic directory of all the files
folder = the folder where the file is found
names = a list consisting of all the file names we want to check

returns the list of all the missing files
"""
def file_not_found(path, folder, names):
  full_path = path + folder + '/'

  files_missing = []

  for i in range(len(names)):
    file_path = full_path + names[i] + '.mp4' # each files ends with a .mp4
    if(not os.path.isfile(file_path)):
      files_missing.append(names[i])

  return files_missing

In [ ]:
files_missing_train = file_not_found(path, 'train', song_names[0: 100]) # checks if there are files missing in the train folder

In [ ]:
files_missing_test = file_not_found(path, 'test', song_names[100: 149]) # checks if there are files missing in the test folder

In [ ]:
print(files_missing_train)
print(files_missing_test)

['Music Delta - Country2.stem', 'Music Delta - Hendrix.stem', 'Music Delta - Punk.stem', 'Music Delta - Reggae.stem', 'Music Delta - Rock.stem', 'Music Delta - Rockabilly.stem']
[]


In [ ]:
"""
a function which gets a list of names and a list of files, and removes the files from the list of names

names = a list with names of files
files = a list with files to be removed from 'names'

return names with the files removed
"""
def name_remove(names, files):
  for i in range(len(files)):
    names.remove(files[i])
  return names

In [ ]:
train_song_names = song_names[0:100] # all the files in the train folder
train_song_names = name_remove(train_song_names, files_missing_train) # removes the missing files
print(len(train_song_names))

94


In [ ]:
test_song_names = song_names[100:150] # all the files in the test folder

# Data augmentation

loads the data into 0.6 segments, converts it to mono and turns into numpy array

In [ ]:
"""
a function which gets a stem file path and returns the file deconstructed 

file_name = the file path to load

returns the file cut into 0.6 seconds segments
"""
def decoding_stems(file_name):
  audio, sr = sp.read_stems(file_name, sample_rate=22050)
  
  swap_ad = np.swapaxes(audio, 1 , 2) # swaps the first and second axis because most libraries execpt num channels in the second axis and the data in the first

  duration = lb.get_duration(swap_ad[1])
  
  duration_length = (duration // 0.6) * 6 # rounds the song duration to be divisilbe by 6 so that each sound segment is in equal length

  return stft_frame(file_name, int(duration_length), 6)

In [ ]:
"""
a function which gets stem file duration and cuts it into 0.6 seconds segments

path = the file path to load
song_duration = how long the stem file is
ld_time = how long each segment should be

returns each segments in a list, after converting to a mono stream
"""
def stft_frame(path, song_duration, ld_time):
  frames = []

  for i in range(0, song_duration, ld_time):
    y, sr = sp.read_stems(path, sample_rate=22050, start=(i / 10), duration=ld_time / 10)  # divides by 10 because you can't loop floats, so we loop with 6 and divide by 10 to get the desired length

    y_swap = np.swapaxes(y, 1, 2)  # swaps the first and second axis because most libraries execpt num channels in the second axis and the data in the first
    frames.append(y_swap)

  return stereo_to_mono(frames)

In [ ]:
"""
a fucntion which gets a list of stereo audio files, and returns each file a a mono 

stereo - a list consisting of stereo audio files

returns each audio files a mono
"""
def stereo_to_mono(stereo):
  mono_frames = []

  for i in range(len(stereo)):
    mono_file = []

    for j in range(len(stereo[i])):
      mono = lb.to_mono(stereo[i][j])
      mono_file.append(mono)

    mono_frames.append(mono_file)

  return mono_frames

In [ ]:
"""
decodes each stem file in the train folder
"""
train_audio_data = []

for i in range(len(train_song_names)):
  
  file_path = path + 'train/' + train_song_names[i] + '.mp4'
  audio_seg = decoding_stems(file_path)
  
  train_audio_data.append(audio_seg)

In [ ]:
"""
decodes each stem file in the train folder
"""
test_audio_data = []

for i in range(len(test_song_names)):
  
  file_path = path + 'test/' + test_song_names[i] + '.mp4'
  audio_seg = decoding_stems(file_path)
  
  test_audio_data.append(audio_seg)

In [ ]:
"""converting the audio lists into numpu arrays"""
train_audio_data = np.array(train_audio_data)
test_audio_data = np.array(test_audio_data)

In [ ]:
print(train_audio_data.shape)
print(test_audio_data.shape)

(94, 11, 5, 13230)
(50, 11, 5, 13230)


# Dataframe creation

consolodates the information into a pandas dataframe and pickels it for future use

In [ ]:
"""
a function which creates a dataframe row with given data

file_name = the name of the audio file
data = the data, which is a list of length = 5, each consisting of a different sound part
folder = what folder the file is in
start = in what time does the segment start in the original sound file
end = in what time does the segment end in the original sound file

returns a dataframe row of the data
"""
def df_add(file_name, data, folder, start, end):
  return pd.DataFrame({0: file_name, 1: [data[0]], 2: [data[1]], 3: [data[2]], 4: [data[3]], 5: [data[4]], 6: folder, 7: start, 8: end})

In [ ]:
"""
makes a dataframe with each row corresponding to a different sound segment in the train folder
"""
df_train = pd.DataFrame()
for i in range(len(train_audio_data)):
  duration = 0
  for j in range(len(train_audio_data[i])):
    df_train = df_train.append(df_add(train_song_names[i], train_audio_data[i][0], 'train', duration, duration+0.6), ignore_index=True, verify_integrity=True)
    duration = duration + 0.6

In [ ]:
"""
makes a dataframe with each row corresponding to a different sound segment in the test folder
"""
df_test = pd.DataFrame()
for i in range(len(test_audio_data)):
  duration = 0
  for j in range(len(test_audio_data[i])):
    df_test = df_test.append(df_add(test_song_names[i], test_audio_data[i][0], 'test', duration, duration+0.6), ignore_index=True, verify_integrity=True)
    duration = duration + 0.6

In [ ]:
"""
concetrates the two dataframe into one coherent dataframe
"""
df_data = pd.concat([df_train, df_test], ignore_index=True, verify_integrity=True)

In [ ]:
df_data.info(memory_usage='deep') # memory data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1584 entries, 0 to 1583
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1584 non-null   object 
 1   1       1584 non-null   object 
 2   2       1584 non-null   object 
 3   3       1584 non-null   object 
 4   4       1584 non-null   object 
 5   5       1584 non-null   object 
 6   6       1584 non-null   object 
 7   7       1584 non-null   float64
 8   8       1584 non-null   float64
dtypes: float64(2), object(7)
memory usage: 1.0 MB


In [ ]:
df_data = df_data.rename(columns={0: 'song_name', 1:'mixture', 2:'drums', 3:'bass', 4:'accompaniment', 5:'vocals', 6:'folder' , 7: 'start', 8:'end'}) # renaming the columns
df_data

,song_name,mixture,drums,bass,accompaniment,vocals,folder,start,end
0,A Classic Education - NightOwl.stem,"[-0.0715077668428421, -0.07427260838449001, -0...","[0.004260942805558443, 0.0010138616780750453, ...","[-0.05491206608712673, -0.05861355736851692, -...","[0.0027123100589960814, 0.0035784956999123096,...","[-0.02593752136453986, -0.022312448360025883, ...",train,0.0,0.6
1,A Classic Education - NightOwl.stem,"[-0.0715077668428421, -0.07427260838449001, -0...","[0.004260942805558443, 0.0010138616780750453, ...","[-0.05491206608712673, -0.05861355736851692, -...","[0.0027123100589960814, 0.0035784956999123096,...","[-0.02593752136453986, -0.022312448360025883, ...",train,0.6,1.2
2,A Classic Education - NightOwl.stem,"[-0.0715077668428421, -0.07427260838449001, -0...","[0.004260942805558443, 0.0010138616780750453, ...","[-0.05491206608712673, -0.05861355736851692, -...","[0.0027123100589960814, 0.0035784956999123096,...","[-0.02593752136453986, -0.022312448360025883, ...",train,1.2,1.8
3,A Classic Education - NightOwl.stem,"[-0.0715077668428421, -0.07427260838449001, -0...","[0.004260942805558443, 0.0010138616780750453, ...","[-0.05491206608712673, -0.05861355736851692, -...","[0.0027123100589960814, 0.0035784956999123096,...","[-0.02593752136453986, -0.022312448360025883, ...",train,1.8,2.4
4,A Classic Education - NightOwl.stem,"[-0.0715077668428421, -0.07427260838449001, -0...","[0.004260942805558443, 0.0010138616780750453, ...","[-0.05491206608712673, -0.05861355736851692, -...","[0.0027123100589960814, 0.0035784956999123096,...","[-0.02593752136453986, -0.022312448360025883, ...",train,2.4,3.0
...,...,...,...,...,...,...,...,...,...
1579,Zeno - Signs.stem,"[-0.006680804304778576, -0.08316835761070251, ...","[0.006809760816395283, 0.009227094938978553, 0...","[-0.036950407549738884, -0.0358741357922554, -...","[-0.009290379006415606, -0.02027385402470827, ...","[0.030881525948643684, -0.03163300175219774, -...",train,3.6,4.2
1580,Zeno - Signs.stem,"[-0.006680804304778576, -0.08316835761070251, ...","[0.006809760816395283, 0.009227094938978553, 0...","[-0.036950407549738884, -0.0358741357922554, -...","[-0.009290379006415606, -0.02027385402470827, ...","[0.030881525948643684, -0.03163300175219774, -...",train,4.2,4.8
1581,Zeno - Signs.stem,"[-0.006680804304778576, -0.08316835761070251, ...","[0.006809760816395283, 0.009227094938978553, 0...","[-0.036950407549738884, -0.0358741357922554, -...","[-0.009290379006415606, -0.02027385402470827, ...","[0.030881525948643684, -0.03163300175219774, -...",train,4.8,5.4
1582,Zeno - Signs.stem,"[-0.006680804304778576, -0.08316835761070251, ...","[0.006809760816395283, 0.009227094938978553, 0...","[-0.036950407549738884, -0.0358741357922554, -...","[-0.009290379006415606, -0.02027385402470827, ...","[0.030881525948643684, -0.03163300175219774, -...",train,5.4,6.0


In [ ]:
df_data.to_pickle(path='/content/drive/MyDrive/Data/MUSDB/df_data.pickle') # pickling the data

# Feature extraction

In [ ]:
df_data = pd.read_pickle('/content/drive/MyDrive/Data/MUSDB/df_data.pickle')
df_data.head()

In [ ]:
def transforming(y_vocals, y_mixture):
  y_vocals = lb.stft(y_vocals, n_fft=2048, win_length=1024, hop_length=512)
  y_mixture = lb.stft(y_mixture, n_fft=2048, win_length=1024, hop_length=512)

  return y_vocals, y_mixture

In [ ]:
y_vocals = []
y_mixture = []
for i in range(len(df_data)):
  a = df_data.at[i, 'mixture']
  b = df_data.at[i, 'vocals']

  v, m = transforming(a, b)
  y_vocals.append(v)
  y_mixture.append(m)

In [ ]:
vocals_stft = np.array(y_vocals)
mixture_stft = np.array(y_mixture)

In [ ]:
print(vocals_stft.shape)
print(mixture_stft.shape)

(1584, 1025, 26)
(1584, 1025, 26)


# Model preparation

In [ ]:
vocals_conv = np.reshape(vocals_stft, newshape=(1584, 1025, 26, 1))
mixture_cov = np.reshape(mixture_stft, newshape=(1584, 1025, 26, 1))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(1025, 26, 1)))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(16, (3,3), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(513))
sgd = SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 1025, 26, 32)      320       
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 1025, 26, 32)      0         
                                                                 
 conv2d_17 (Conv2D)          (None, 1025, 26, 16)      4624      
                                                                 
 leaky_re_lu_21 (LeakyReLU)  (None, 1025, 26, 16)      0         
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 341, 8, 16)       0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 341, 8, 16)        0         
                                                      